In [8]:
cmmds="""
pip install protobuf==3.20.0
export PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python
git clone https://github.com/PKU-YuanGroup/Video-LLaVA.git
cd Video-LLaVA

pip install --upgrade pip  # enable PEP 660 support
pip install -e .
pip install -e ".[train]"
pip install flash-attn --no-build-isolation
pip install decord opencv-python git+https://github.com/facebookresearch/pytorchvideo.git@28fe037d2126
pip install pytube

"""
with open('cmmds.sh','w+') as fp:
    fp.writelines(cmmds)

!chmod +x cmmds.sh
!./cmmds.sh

Cloning into 'Video-LLaVA'...
remote: Enumerating objects: 948, done.
remote: Counting objects: 100% (325/325), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 948 (delta 266), reused 175 (delta 174), pack-reused 623
Receiving objects: 100% (948/948), 115.33 MiB | 15.63 MiB/s, done.
Resolving deltas: 100% (505/505), done.
Obtaining file:///content/Video-LLaVA
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for videollava (pyproject.toml) ... done
  Created wheel for videollava: filename=videollava-1.0.0-0.editable-py3-none-any.whl size=12746 sha256=a4bcf90fa425e8d6be438be79645e5bec3fed6e09a8c80faf5e9c987b4846487
  Stored in directory: /tmp/pip-ephem-wheel-cache-vk1o_vrs/wheels/2c/aa/87/e87ec400fd8d50e7ca7616464e202d06a54e73967293159deb
Successf

In [2]:
import torch
import time
import os
from videollava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from videollava.conversation import conv_templates, SeparatorStyle
from videollava.model.builder import load_pretrained_model
from videollava.utils import disable_torch_init
from videollava.mm_utils import tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria
def model_predict(tokenizer, model, processor,video,inp):
    video_processor = processor['video']
    conv_mode = "llava_v1"
    conv = conv_templates[conv_mode].copy()
    roles = conv.roles
    tokenizer, model, processor
    video_tensor = video_processor(video, return_tensors='pt')['pixel_values']
    if type(video_tensor) is list:
        tensor = [video.to(model.device, dtype=torch.float16) for video in video_tensor]
    else:
        tensor = video_tensor.to(model.device, dtype=torch.float16)

    print(f"{roles[1]}: {inp}")
    inp = ' '.join([DEFAULT_IMAGE_TOKEN] * model.get_video_tower().config.num_frames) + '\n' + inp
    conv.append_message(conv.roles[0], inp)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()
    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()
    stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
    keywords = [stop_str]
    stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)

    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=tensor,
            do_sample=True,
            temperature=0.1,
            max_new_tokens=1024,
            use_cache=True,
            stopping_criteria=[stopping_criteria])

    outputs = tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()
    print(outputs)
    return outputs




[2024-02-21 15:10:16,090] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


In [12]:
from pytube import YouTube
import time
import os
def get_video(link):
  yt = YouTube(link)
  stream = yt.streams.get_highest_resolution()
  stream.download('/content/videos')
def load_model():
    model_path = 'LanguageBind/Video-LLaVA-7B'
    cache_dir = 'cache_dir'

    device = 'cuda'
    load_4bit, load_8bit = True, False
    model_name = get_model_name_from_path(model_path)
    tokenizer, model, processor, _ = load_pretrained_model(model_path, None, model_name, load_8bit, load_4bit, device=device, cache_dir=cache_dir)
    return (tokenizer,model,processor)

if __name__ == '__main__':
  disable_torch_init()
  prompts = [
    "Can you provide a summary of the video?",
    "Describe the events unfolding in the video.",
    "Give me an overview of what's going on in the video.",
    "Who are the main characters in the video?",
    "What is the setting of the video?",
    "What is the main conflict or problem presented in the video?",
    "What are the key events leading up to this point in the video?",
    "Can you describe any notable actions or dialogue in the video?",
]

  re={'name':[],'time':[],"prompt":[],'output':[]}
  # tokenizer,model,processor=load_model()
  videos=['https://www.youtube.com/shorts/7rN484kTzMQ',
'https://www.youtube.com/shorts/ke7icLLG3_Q',
'https://www.youtube.com/shorts/BrZPULieSNM',
'https://www.youtube.com/shorts/QlogSrdVaJs',
'https://www.youtube.com/watch?v=XMKg-8OZRGQ',
'https://www.youtube.com/watch?v=feqafo-7Dok',
'https://www.youtube.com/watch?v=UoeZEUH6uu8']
  for i in videos:
      get_video(i)
  for i in os.listdir('/content/videos/'):
    for j in prompts:
      re['prompt'].append(j)
      re["name"].append(i)
      start=time.time()
      print(i)
      re['output'].append(model_predict(tokenizer, model, processor,os.path.join('/content/videos',i),j))
      end=time.time()-start
      re['time'].append(end)


The Government is After Howard  The Big Bang Theory.mp4
ASSISTANT: Can you provide a summary of the video?
Sure! The video shows a group of people sitting in a car, with a man driving and a woman sitting next to him. They are all wearing glasses and the man is talking on a cell phone. The woman is also talking on the phone, and the man is looking at her.</s>
The Government is After Howard  The Big Bang Theory.mp4
ASSISTANT: Describe the events unfolding in the video.
The video shows a group of people sitting in a car, with a man driving and a woman sitting next to him. They are seen talking and laughing while driving down the road.</s>
The Government is After Howard  The Big Bang Theory.mp4
ASSISTANT: Give me an overview of what's going on in the video.
The video shows a group of people sitting in a car, talking and laughing. They are seen driving around and eventually stop to eat at a restaurant.</s>
The Government is After Howard  The Big Bang Theory.mp4
ASSISTANT: Who are the main c

In [15]:
import pandas as pd
result_df=pd.DataFrame(re)

In [16]:
result_df.to_csv('result_df.csv')